In [ ]:
#  #########################################################################
#  #                           IN THE NAME OF ALLAH                        #
#  #     OPTIMIZATION ANALYSIS WEB HEIGHT NONPRISMATIC I SECTION CLOUMN    #
#  #     WITH FINITE PRISMATIC CLOUMN WITH NONLINEAR PUSHOVER ANALYSIS     #
#  #                         NEWTON-RAPHSON METHOD                         #
#  #    MODELING OF NONPRISMATIC ELEMENT WITH MULTI PRISMATIC ELEMENTS     #
#  #-----------------------------------------------------------------------#
#  #              THIS PROGRAM WRITTEN BY SALAR DELAVAR QASHQAI            #
#  #                   EMAIL: salar.d.ghashghaei@gmail.com                 #
#  #########################################################################

In [ ]:
#import the os module
import os
import math
import time

In [ ]:
#to create a directory at specified path with name "Data"
os.mkdir('C:\\OPENSEESPY_SALAR')
#this will create the directory with name 'Data' and will update it when we rerun the analysis,
# otherwise we have to keep deleting the old 'Data' Folder
dir = "C:\\OPENSEESPY_SALAR\\OPENSEESPY_DATA"
if not os.path.exists(dir):
    os.makedirs(dir)

In [ ]:
# Your existing code
def MAXABS_FUN(DATA_FILE):
    import numpy as np
    # Read and process displacement data
    NameFiles = DATA_FILE
    filename = f"{NameFiles}.txt"
    D = np.loadtxt(filename)
    #print(D)
    MAXABS = np.max(np.abs([D[:, 3]]))
    #print("MAX. ABS. :", MAXABS)
    return MAXABS

In [ ]:
# pip install openseespy

In [ ]:
# OUTPUT DATA ADDRESS:
SALAR_DIR = 'C:/OPENSEESPY_SALAR/OPENSEESPY_DATA/';

In [ ]:
#
#    ^Y
#    |
#    2       __ 
#    |          | 
#    |          |
#    |          |
#  (1)       LCol
#    |          |
#    |          |
#    |          |
#  =1=      _|_  -------->X
#

# SET UP ----------------------------------------------------------------------------


def PUSHOVER_ANALYSIS(hw, tw,tf, bf, LCol, N, Weight, DMAX):
    import openseespy.opensees as op
    op.wipe()
    op.model('basic', '-ndm', 2, '-ndf', 3) 
    PCol =Weight  # nodal dead-load weight per column
    g =  9810 # mm/s^2
    Mass =  PCol/g
    ll = LCol / (N - 1)
    IDctrlNode = N
    IDctrlDOF = 1
    
    # MATERIAL parameters -------------------------------------------------------------------
    IDreinf = 1; 				# material ID tag -- steel 
    Fy = 4000			# STEEL yield stress
    Cy = 0.02			# STEEL yield stress
    Es = Fy/Cy				# modulus of steel
    Bs = 0.01				# strain-hardening ratio 
    R0 = 18.0				# control the transition from elastic to plastic branches
    cR1 = 0.925				# control the transition from elastic to plastic branches
    cR2 = 0.15				# control the transition from elastic to plastic branches
    op.uniaxialMaterial('Steel02', IDreinf, Fy, Es, Bs, R0,cR1,cR2) # build reinforcement material
    # ---------------------------------------------------------------------------------------
    hwtop = 0.5 * hw # Top column web height
    # nodal coordinates:
    op.node(1, 0.0, 0.0) # node#, X, Y
    # Single point constraints -- Boundary Conditions
    op.fix(1, 1, 1, 1) # node DX DY RZ
    for i in range(2, N + 1, 1):
        op.node(i, 0.0, ll * (i-1))
        ColSecTag = i-1			# assign a tag number to the column section
        ColTransfTag = i-1
        eleTag = i-1

        # FIBER SECTION properties -------------------------------------------------------------
        # symmetric section
        #                        y
        #                        ^
        #                        |     
        #              _____________________    --   --
        #             |_________   ________|    |    -- tf
        #                      |  |             |
        #                      |  |             |
        #    z <---       hw   |tw|             H
        #                      |  |             |
        #              ________|  |_________    |
        #             |____________________|    |    -- tf
        #                                      --    --
        #             |-------- bf --------|
        #
        # STEEL I SECTION: 
        HW =  hw + ((hwtop - hw) / LCol) * ll * (i-2) # Varying web height of nonprismatic section
        coverY = (HW + tf) / 2.0
        coverZ = tw / 2.0
        coreY = coverY - tf
        coreZ02 = bf / 2.0

        nfCoreY = 15;			# number of fibers for steel in y-direction
        nfCoreZ = 5;			# number of fibers for steel in z-direction


        op.section('Fiber', ColSecTag)
        # Define the core patch
        op.patch('quad', IDreinf, nfCoreZ, nfCoreY, -coverY, coreZ02, -coverY, -coreZ02, -coreY,-coreZ02, coreY, coreZ02) # TOP FLANGE
        op.patch('quad', IDreinf, nfCoreZ, nfCoreY, -coreY,coverZ, -coreY,-coverZ, coreY,-coverZ, coreY, coverZ) # MIDDLW WEB
        op.patch('quad', IDreinf, nfCoreZ, nfCoreY, coreY, coreZ02, -coreY, coreZ02, coverY,-coreZ02, coreY, coreZ02) # TOP FLANGE

        op.geomTransf('Linear', ColTransfTag)
        numIntgrPts = 5
        op.element('nonlinearBeamColumn', eleTag, i-1, i, numIntgrPts, ColSecTag, 1)


    #import InelasticFiberSection
    op.recorder('Node', '-file', f"{SALAR_DIR}DTH.txt",'-time', '-node', N, '-dof', 1,2,3, 'disp')# Displacement Time History
    op.recorder('Node', '-file', f"{SALAR_DIR}BTH.txt",'-time', '-node', 1, '-dof', 1,2,3, 'reaction')# Base Shear Time History

    # node#, Mx My Mz, Mass=Weight/g, neglect rotational inertia at nodes
    op.mass(N, Mass, 1e-9, 0.0)
    
    #defining gravity loads
    op.timeSeries('Linear', 1)
    op.pattern('Plain', 1, 1)
    op.load(N, 0.0, -PCol, 0.0)

    Tol = 1e-8 # convergence tolerance for test
    Iter = 1000000# convergence iteration for test
    NstepGravity = 10
    DGravity = 1 / NstepGravity
    op.integrator('LoadControl', DGravity) # determine the next time step for an analysis
    op.numberer('Plain') # renumber dof's to minimize band-width (optimization), if you want to
    op.system('BandGeneral') # how to store and solve the system of equations in the analysis
    op.constraints('Plain') # how it handles boundary conditions
    op.test('NormDispIncr', Tol, Iter) # determine if convergence has been achieved at the end of an iteration step
    op.algorithm('Newton') # use Newton's solution algorithm: updates tangent stiffness at every iteration
    op.analysis('Static') # define type of analysis static or transient
    op.analyze(NstepGravity) # apply gravity

    op.loadConst('-time', 0.0) #maintain constant gravity loads and reset time to zero
    #print('Model Built')
    
    Dincr = 0.001 * DMAX
    Hload = Weight#Weight
    maxNumIter = 1000
    tol = 1e-8

    op.timeSeries('Linear', 2)
    op.pattern('Plain', 200, 2)
    op.load(N, Hload, 0.0, 0.0)

    op.wipeAnalysis()
    op.constraints('Plain')
    op.numberer('Plain')
    op.system('BandGeneral')
    op.test('EnergyIncr', Tol, maxNumIter)
    op.algorithm('Newton')

    op.integrator('DisplacementControl', IDctrlNode, IDctrlDOF, Dincr)
    op.analysis('Static')


    Nsteps =  int(DMAX/ Dincr)

    ok = op.analyze(Nsteps)
    #print(ok)

    # for gravity analysis, load control is fine, 0.1 is the load factor increment (http://opensees.berkeley.edu/wiki/index.php/Load_Control)

    test = {1:'NormDispIncr', 2: 'RelativeEnergyIncr', 4: 'RelativeNormUnbalance',5: 'RelativeNormDispIncr', 6: 'NormUnbalance'}
    algorithm = {1:'KrylovNewton', 2: 'SecantNewton' , 4: 'RaphsonNewton',5: 'PeriodicNewton', 6: 'BFGS', 7: 'Broyden', 8: 'NewtonLineSearch'}

    for i in test:
        for j in algorithm:

            if ok != 0:
                if j < 4:
                    op.algorithm(algorithm[j], '-initial')

                else:
                    op.algorithm(algorithm[j])

                op.test(test[i], Tol, 1000)
                ok = op.analyze(Nsteps)                            
                #print(test[i], algorithm[j], ok)             
                if ok == 0:
                    break
            else:
                continue

    #u2 = op.nodeDisp(2, 1)
    #print("u2 = ", u2)
    print('Pushover Done')
    op.wipe()
    


In [ ]:
### --------------------------------------------------------------
###          NONPRISMATIC I SECTION WEB HEIGHT OPTIMIZATION
### --------------------------------------------------------------

# define section geometry
LCol = 3000.0 # [mm] column length
hw = 350 # [mm] Section Web Hight 
tw = 10 # [mm] Section Web Thickness
tf = 10 # [mm] Section Flange Thickness
bf = 110 # [mm] Section Flange Width

N = 50 # Columns Node Count

Weight = 100000.0 # [N] superstructure weight

X = hw # Intial Guess for I Section web geight
ESP = 1e-3 # Finite difference derivative Convergence Tolerance
TOLERANCE = 1e-4 # Convergence Tolerance
RESIDUAL = 100 # Convergence Residual 
IT = 0 # Intial Iteration
ITMAX = 100000 # Max. Iteration
DMAX = 10 # [mm] Max. Pushover Incremental Displacement
TARGET_BASEMOMENT = 2.5000e+06 # [N] Target Demand Max. Abs. Base Moment
DATA_FILE ='C:\OPENSEESPY_SALAR\OPENSEESPY_DATA\BTH'  # MAX BASE MOMENT

TARGET_DISP = 7 # [mm] Target Demand Max. Abs. Displacement
#DATA_FILE ='C:\OPENSEESPY_SALAR\OPENSEESPY_DATA\DTH'  # MAX DISPLACEMENT

# monitor cpu time
import time
t = time.localtime()
current_time = time.strftime("%H:%M:%S", t)
print(f"Current time (HH:MM:SS): {current_time}\n\n")

### FIND THE OPTIMUM VALUE 
while (RESIDUAL > TOLERANCE):
    PUSHOVER_ANALYSIS(X, tw, tf, bf, LCol, N, Weight, DMAX)
    time.sleep(10);# Sleep for 10 seconds
    F = MAXABS_FUN(DATA_FILE) - TARGET_BASEMOMENT
    print('Current Max. Abs. Base Moment: ', MAXABS_FUN(DATA_FILE))
    #print('F: ', F)
    # Evaluate at Xmain and Fmin
    Xmin = X - ESP
    PUSHOVER_ANALYSIS(Xmin, tw , tf, bf, LCol, N, Weight, DMAX)
    time.sleep(10);# Sleep for 10 seconds
    Fmin = MAXABS_FUN(DATA_FILE) - TARGET_BASEMOMENT
    #print('Fmin: ', Fmin)
    # Evaluate at Xmax and Fmax
    Xmax = X + ESP
    PUSHOVER_ANALYSIS(Xmax, tw, tf, bf, LCol, N, Weight, DMAX)
    time.sleep(10);# Sleep for 10 seconds
    Fmax = MAXABS_FUN(DATA_FILE) - TARGET_BASEMOMENT
    #print('Fmax: ', Fmax)
    DF = (Fmax-Fmin)/(2*ESP);# Calculate the Finite difference derivative of F
    #print('DF: ', DF)
    DX = F / DF; # Calculate dx
    RESIDUAL = abs(DX); # Calculate residual
    print('RESIDUAL: ', RESIDUAL, ' - WEB HEIGHT: ', X)
    X -= DX; # update X
    IT += 1; # update iteration
    if IT == ITMAX:
        print("\t\t Iteration reached to Max. Iteration")
        print("\t\t Change ESP and TOLERANCE for better Convergence")
        X = -1
        break;
    if RESIDUAL < TOLERANCE:
        print(f'\t\t Optimum Section Web Height:   {X:.4f}')
        print(f'\t\t Iteration Counts:             {IT}')
        print(f'\t\t Convergence Residual:         {RESIDUAL:.10e}')
    #print(X)
    
t = time.localtime()
current_time = time.strftime("%H:%M:%S", t)
print(f"Current time (HH:MM:SS): {current_time}\n\n")

In [ ]:
#### PLOT COLUMN LENGTH AND WEB HEIGHT NONPRISMATIC I SECTION CLOUMN WITH FINITE PRISMATIC CLOUMN
hw = X # Optimum Web Height
z = []
x = []
l = LCol / N
hwtop = 0.5 * hw
# Top column web height
for i in range(2, N+1, 1):
    L = l * (i-1)
    x.append(L)
    HW =  hw + ((hwtop - hw) / LCol) * L # Varying web height of nonprismatic section
    z.append(HW)
    print(f' Element: {i-1} Column Length: {L:.2f} - Web Height: {HW:.2f}')

import matplotlib.pyplot as plt


# Plot shear force and bending moment
plt.figure(figsize=(12, 8))
plt.plot(x, z,color='black')
plt.xlabel('COLUMN LENGTH')
plt.ylabel('WEB HEITHT')
plt.title(f'COLUMN LENGTH AND WEB HEIGHT NONPRISMATIC I SECTION WITH FINITE {N} PRISMATIC SECTIONS')
plt.grid()
plt.show()
    